In [18]:
import sys
sys.path.append('/Users/markrice/Documents/GitHub/RGH_NLP/')

import re
import pandas as pd
import numpy as np
from DataTools import language_datatools as lang

In [9]:
# network_csv = pd.read_csv(input('Enter Filename:'))  #Read in file,error checking will come later. For now, I trust me.
network_csv = pd.read_csv('/Users/markrice/OneDrive/RGH/NLP_PS/HPG_HPA.csv')
network_csv

,RelationSymbolicName,ObjectType,Effect,Mechanism,RelationNumberOfReferences,RelationConfidenceLevel,msrc
0,MolSynthesis: FST ---> progesterone,MolSynthesis,NaN,NaN,10,3,"As for ID{4000000,4108202=steroidogenesis}, ID..."
1,positive MolSynthesis: epinephrine --+> proges...,MolSynthesis,positive,NaN,7,3,It was reported that ID{1271077=epinephrine} a...
2,positive MolSynthesis: Luteinizing hormone --+...,MolSynthesis,positive,NaN,53,3,Social contact stimulates the secretion of ID{...
3,negative MolSynthesis: cortisol ---| ACTH,MolSynthesis,negative,NaN,46,3,"Finally, it has been previously shown that ID{..."
4,positive MolSynthesis: CRH --+> cortisol,MolSynthesis,positive,NaN,73,3,"ID{10000000,8802722=Placental} ID{1392=CRH} ma..."
5,positive MolSynthesis: cortisol --+> epinephrine,MolSynthesis,positive,NaN,9,3,ID{1100010=Cortisol} promotes the synthesis of...
6,negative MolSynthesis: CRH ---| progesterone,MolSynthesis,negative,NaN,15,3,Because ID{1392=corticotropin-releasing factor...
7,positive MolSynthesis: norepinephrine --+> pro...,MolSynthesis,positive,NaN,10,3,ID{1267744=norepinephrine} acts on ID{154=β2-a...
8,positive MolSynthesis: cortisol --+> norepinep...,MolSynthesis,positive,NaN,6,3,"The rise in ID{10000000,6800052=plasma} ID{110..."
9,positive MolSynthesis: FSH --+> estrogen,MolSynthesis,positive,NaN,133,3,ID{3000051=follicle-stimulating hormone} stimu...


In [10]:
#Conform Data
network = pd.DataFrame(columns=['Source', 'Target', 'Data'])

for i in network_csv.index:
    strLine = network_csv.iloc[i][0]
    m = re.search('(?<=: ).*', strLine)
    strLine = m.group()
    m = re.search('.*(?= --)', strLine)
    df_source = m.group()
    m = re.search('(?<=[>|] ).*', strLine)
    df_target = m.group()
    df_data = network_csv.iloc[i][6]

    network.loc[i] = [df_source, df_target, df_data]

In [15]:
network['Reference_Count'] = network_csv.RelationNumberOfReferences
network2 = network

#Consolodate Duplicates and  Data
network = network.groupby(['Source', 'Target'])['Data'].apply(' '.join).reset_index()
for idx, row in network.iterrows():
    strLine = row.Data
    strLine = re.sub('\|',' ',strLine)
    strLine = re.sub('\"', '', strLine)
    strLine = re.sub('  ', ' ', strLine)
    row.Data = strLine

In [17]:
network

,Source,Target,Data,Reference_Count
0,ACTH,CRH,Blocking ID{1392=CRH}-stimulated increases in ...,10
1,ACTH,GNRH1,The increased levels of ID{1392=CRH} and ID{14...,7
2,ACTH,cortisol,"Besides, different types of ID{9000000,9016100...",53
3,ACTH,epinephrine,"CRF then causes the ID{10000000,8803038=pituit...",46
4,ACTH,progesterone,ID{1409770=Adrenocorticotropic hormone} stimul...,73
5,AVP,ACTH,ID{551=AVP} binding to ID{553=V1bR} in the ID{...,9
6,AVP,CRH,ID{551=AVP} may also directly increase secreti...,15
7,AVP,KISS1,Several lines of evidence have suggested that ...,10
8,AVP,OXT,We mention here that ID{1267744=norepinephrine...,6
9,AVP,cortisol,Several studies have demonstrated that ID{551=...,133


In [24]:
lang.network_score(network)

FileNotFoundError: [Errno 2] No such file or directory: 'positive.yml'

In [14]:
network[network['Polarity'] != 'Unknown'].reset_index().drop('Data',1).to_csv(path_or_buf='HPG_HPA_Sentiment.csv',
                                                                              columns= ['Source','Target','Polarity'],index=False)